# Experimentation Guided Example



Over the past several assignments we've walked through many of the elements of good experimentation, covering statistical tests, research proposals, and more. But things are always a little different in the wild, so now we're going to put many of these elements together and walk through an example of what an experiment might look like when you're actually running it. The goal here is to see how the pieces fit together.

This is an interactive guided example. Changes you make to the version of this notebook on Thinkful's servers won't stick around. Make a copy of this notebook right now and save it to your local environment so that you'll have your own version, then switch over to that version.

## Setting the Stage

So let's talk about the scenario. For this example we're going to use a generic marketing scenario focused on email conversion. Many companies in a variety of industries run email campaigns to get people to sign up for a service.  Even if your interest is not in marketing, any experiment designed to elicit a discrete response will be similarly constructed.

Our theoretical company is a social networking site, called BlueBird. BlueBird has been using the same advertising email for years, and some at the company are starting to feel that it's getting a little stale. Ever the data driven individual, you propose an experiment. The marketing department draws up a new version of the email, and you'll conduct an A/B test comparing the two emails.

## Before Rollout

You may want to roll out the experiment right away, but there is so much to do beforehand! Getting an experiment ready to go out the door is serious business. It will make everything that comes after easier and make sure you're making well thought out decisions all the way through.

Now the research proposal or similar document can play a huge role in this prep process. That should contain a detailed description of the motivation of the experiment, what exactly the change is, and a plan for rollout. For this experiment, let's highlight a few elements of the research proposal here, rather than report the whole document.

**Copy of Both Emails**. This is useful so that anyone looking into the experiment can use the research proposal to actually see what happens.

**Rollout Plan**. Knowing how many people you intend to treat with your new email and how long this is going to take is important. It's not only a key aspect of the experiment itself, but also something you're likely to be asked about by other people who care about the outcome. Often times, though 50:50 rollouts are the fastest, a gradual rollout will be used to ensure that there are no negative consequences of the experiment or if things are broken or severely degraded it can be rolled back quickly. In this case we'll do two weeks at 10% and then two weeks at 50%. That will lead to about 30,000+ in treatment, which is a nice sized sample.

_As a note, if you want to do a more rigorous calculation of sample size, you can do power calculations before hand._ Power _is the probability of rejecting a false null hypothesis. This uses the formula for t-test to see what effect would be detectable at a given sample size [also referred to as 'N'](https://onlinecourses.science.psu.edu/stat500/node/46)_.

**Success Metric**. It may seem obvious, but defining success before rolling out an experiment is a key thing to do. Usually this means picking the key metric of interest and setting the degree of significance you want to have. For a t-test this is typically a p-value of .05, and that's what we'll do here. Our success metric will be signups (also called conversions).

**Secondary Metrics**. It's easier to evaluate success if you have one success metric, but that doesn't mean that's all you look at. There can be many other metrics of interest, and monitoring them will ensure that you're not having an adverse effect on other things that matter. In this case let's make time from email to signup a secondary metric.

**Other Things to Do...**

You should also _test the infrastructure_, if any, before rolling out. In this case that means making sure the email is set up properly to be sent out when the experiment turns on. If it's manual, that's pretty easy (make sure it's formatted properly and the email is generally deliverable). If you use a service, sending some test emails to test accounts is probably a good idea. This can be either an engineering or a data science task, depending on complexity.

Make sure you have a _method for randomly sampling subjects_, and know if people stay in test or control permanently. This can matter for something like emails if people receive multiple messages. Should they only receive the new style of email or should each email be random? For simplicity, we'll assume each individual is only emailed once.

This is also a good time to check if there is any potential _collision with other events or experiments_. You want to run experiments in times that are representative of standard operations (unless your experiment is explicitly about how to handle an extreme situation). If you were testing marketing strategies for football betting behavior right before the Super Bowl, for example, you might see some weird behaviors that would make your findings not broadly applicable. You also don't want to test two things that are related at the same time.

Another consideration is _segmentation_ of your sample. Sometimes you don't want to test on the entire population. Maybe you'll pick specific cities (in which case it may be appropriate to sample those in a representative fashion) or test a specific age group or tenure of user. Make these decisions early as they will help inform later analysis and possibly how the experiment is functionally set up. For here we'll just assume we're small enough to test on the entire population.

Finally, it is good practice to give everyone a chance to _review the research proposal and provide feedback_. Sometimes this is done through a kickoff meeting while other companies may have less formal processes. Make sure everyone involved is aware of the plan and bought in before rolling out.

After all of this is done, and the boxes are all checked, you're ready to roll. Let's say we flipped the switch and turned the experiment on at 10% on April 1, 2016.

## Two Weeks In

Of course, you would monitor the experiment throughout its time in the wild, but two weeks in when considering turning up the proportion is a good time to do some analytics to see how things are performing before you roll forward to 50%.

We've [got a file](https://github.com/Thinkful-Ed/data-201-resources/blob/master/1.4.11_experimentation_guided_example/experiment_data_2_wks.csv) for what the data looks like up to this point. Save this CSV to your local environment in the same folder as your notebook as `experiment_data_2_wks.csv`.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
PATH = 'https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/1.4.11_experimentation_guided_example/experiment_data_2_wks.csv'
experiment_data = pd.read_csv(PATH)

In [3]:
experiment_data.head(10)

,treatment_date,test,is_signed_up,signup_date
0,2016-04-01,0,0,NaN
1,2016-04-01,0,0,NaN
2,2016-04-01,0,0,NaN
3,2016-04-01,0,0,NaN
4,2016-04-01,0,0,NaN
5,2016-04-01,0,0,NaN
6,2016-04-01,1,1,2016-04-02
7,2016-04-01,0,0,NaN
8,2016-04-01,0,0,NaN
9,2016-04-01,0,0,NaN


OK, so the data seems to have a few columns. `treatment_date` shows the date of the email being sent. `test` is a binary indicator if they were in test or control. `is_signed_up` shows if they successfully signed up. `signup_date` is the date of signup. Simple enough.

But how are things going? Let's write some code to see.

__Answer the questions below about this data to see how the experiment is developing__

In [42]:
# How large is each group?
print('test sample size:', len(experiment_data[experiment_data.test == 1]))
print('control sample size:', len(experiment_data[experiment_data.test == 0]))
print('test proportion:', experiment_data.test.mean())
print()

# What is the signup rate of test and control?
print('test signup rate:', experiment_data[experiment_data.test == 1].is_signed_up.mean())
print('control signup rate:', experiment_data[experiment_data.test == 0].is_signed_up.mean())

# Are the test and control groups significantly different?
stats.ttest_ind(experiment_data[experiment_data.test == 1].is_signed_up,
                experiment_data[experiment_data.test == 0].is_signed_up)

test sample size: 3372
control sample size: 30241
test proportion: 0.10031832921786214

test signup rate: 0.13701067615658363
control signup rate: 0.14708508316523924


Ttest_indResult(statistic=-1.5711354292574262, pvalue=0.11616060761145311)

So the groups are not significantly different, and indications do not seem great. The test signup rate seems lower than the old message. Can you see any reason why?

**Use the cell below to write some code to investigate. It might be worth looking at the secondary metric:**

In [50]:
experiment_data[experiment_data.test == 1].signup_date

6        2016-04-02
22              NaN
23              NaN
30              NaN
84              NaN
91              NaN
92              NaN
99              NaN
101             NaN
107             NaN
111      2016-04-13
133      2016-04-06
138             NaN
145             NaN
173      2016-04-06
176             NaN
185      2016-04-03
205             NaN
208             NaN
222             NaN
225      2016-04-07
226             NaN
236             NaN
241             NaN
253             NaN
255             NaN
261             NaN
275             NaN
278             NaN
282             NaN
            ...    
33282           NaN
33289           NaN
33291           NaN
33295           NaN
33317           NaN
33318           NaN
33320           NaN
33325           NaN
33334           NaN
33351           NaN
33357           NaN
33358           NaN
33382           NaN
33399           NaN
33423           NaN
33436           NaN
33440           NaN
33459           NaN
33465           NaN


In [51]:
experiment_data[experiment_data.test == 1].treatment_date

6        2016-04-01
22       2016-04-01
23       2016-04-01
30       2016-04-01
84       2016-04-01
91       2016-04-01
92       2016-04-01
99       2016-04-01
101      2016-04-01
107      2016-04-01
111      2016-04-01
133      2016-04-01
138      2016-04-01
145      2016-04-01
173      2016-04-01
176      2016-04-01
185      2016-04-01
205      2016-04-01
208      2016-04-01
222      2016-04-01
225      2016-04-01
226      2016-04-01
236      2016-04-01
241      2016-04-01
253      2016-04-01
255      2016-04-01
261      2016-04-01
275      2016-04-01
278      2016-04-01
282      2016-04-01
            ...    
33282    2016-04-14
33289    2016-04-14
33291    2016-04-14
33295    2016-04-14
33317    2016-04-14
33318    2016-04-14
33320    2016-04-14
33325    2016-04-14
33334    2016-04-14
33351    2016-04-14
33357    2016-04-14
33358    2016-04-14
33382    2016-04-14
33399    2016-04-14
33423    2016-04-14
33436    2016-04-14
33440    2016-04-14
33459    2016-04-14
33465    2016-04-14
